In [60]:
import requests_cache

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import os

In [61]:
requests_cache.install_cache('demo_cache',expire_after=None,allowable_methods=['GET'])
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [62]:
link = "https://www.worldometers.info/coronavirus/"

driver.get(link)

In [63]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [64]:
header_by_day = ["#","Country","Date","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered","NewRecovered",
"ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

header_by_week = ["#", "Country", "Cases in the last 7 days", "Cases in the preceding 7 days",
"Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", 
"Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"]

In [65]:
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

rows_past = []
rows_past_modify = []
rows_yesterday = []
rows_two_days_ago = []
count = 0
row_count = 226

for n in row_total:
    n = n.get_attribute('textContent')
    if count == row_count:
        rows_past.append(n.split("\n"))
    else:
        count+=1

for n in rows_past:
    n.pop(0)
    n = n[: len(n) - 6]
    rows_past_modify.append(n)

rows_yesterday,rows_two_days_ago = split_list(rows_past_modify)

rows_yesterday = [ele for ele in rows_yesterday if ele != []]
rows_two_days_ago = [ele for ele in rows_two_days_ago if ele != []]

In [66]:
today = date.today()
yesterday = (today - timedelta(1)).strftime("%d-%m-%Y")
two_days_ago = (today - timedelta(2)).strftime("%d-%m-%Y")

for row in rows_yesterday:

    row.insert(2,yesterday)
for row in rows_two_days_ago:

    row.insert(2,two_days_ago)

total_covid = rows_two_days_ago + rows_yesterday 

Check xem có dòng nào bị dư giá trị rỗng không, nếu có cần chạy lại từ đầu

In [67]:
for i in total_covid:
    if len(i) > 17:
        if i[-1] == '' or i[-1] == ' ':
            i.pop(-1)
        else:
            print(i)

In [68]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)

In [69]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)
if not os.path.isfile('Corona_by_day.tsv'):
    df_covid = pd.DataFrame(data=total_covid,columns=header_by_day)
    df_covid.to_csv("Corona_by_day.tsv", sep = "\t", mode = "a", index = False)
else:
    df_covid.to_csv("Corona_by_day.tsv", sep = "\t", mode = "a", index = False,header=False)

Crawl weekly data

In [70]:
link = "https://www.worldometers.info/coronavirus/weekly-trends/#weekly_table"
driver.get(link)
weekly_data = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

In [71]:
for n in weekly_data:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    if len(n) != len(header_by_week):
        print("Error Header in",n)

In [72]:
row_week_trend = []

for n in weekly_data:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    row_week_trend.append(n)
    if len(n) != len(header_by_week):
        print("Error Header in",n)

In [73]:
df_covid_by_week = pd.DataFrame(data=row_week_trend,columns=header_by_week)
df_covid_by_week.to_csv("Corona_by_week.tsv", sep = "\t", mode = "w", index = False)